In [ ]:
import numpy as np

import sys
import os
import json

import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

from tensorflow.keras.utils import Progbar
from sklearn.manifold import TSNE

In [ ]:
%matplotlib inline

In [ ]:
print('Python version: %s' % sys.version)
print('Numpy version: %s' % np.__version__)
print('Tensorflow version: %s' % tf.__version__)
print('Tensorflow datasets version: %s' % (tfds.__version__))

# MNIST

In [ ]:
(mnist_train, mnist_test), mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True, shuffle_files=True, split=['train', 'test'])
mnist_info

Определим функцию, возвращающую преобразование пары (картинка, номер класса) в нормированную картинку, плюс, возможно, номер класса, в зависимости от настроек.

In [ ]:
def rescaler(scale=255, with_label=True):
    
    def rescale(x, y):
        return 1 - x / scale
    
    def rescale_and_label(x, y):
        return rescale(x), y
    
    return rescale_and_label if with_label else rescale

Несколько примеров из MNIST

In [ ]:
plt.figure(figsize=(10, 7))
rescale = rescaler(255, with_label=False)
for n, (x, y) in enumerate(mnist_train.take(15)):
    plt.subplot(3, 5, n + 1)
    plt.imshow(rescale(x, y), cmap='gray')
    plt.axis('off')
    plt.title(y.numpy())
plt.show()

# Super Resolution

Цель этого задания - познакомиться с операцией деконволюции на примере фильтра, который увеличивает изображения в 2 раза с помощью билинейной интерполяции. Вместо того, чтобы выводить его из теоретических соображений, мы построим соответствующую модель и обучим её увеличивать изображения.

## Задание 1.а

Датасет MNIST состоит из пар `(X, y)`, где `X` это монохромное изображение 28x28, `y` это класс. Необходимо написать фукнцию, преобразующую эту пару в пару `(x, X)`, где `x` это уменьшенное в 2 раза изображение цифры, т.е. монохромное изображение 14x14. При этом значение пикселей в изображениях должны быть отнормированы на интервал `[0,1]`. Также если хочется видеть черные цифры на белом фоне, а не наоборот, можно инвертировать цвета: `x -> 1-x`.

Определяем по аналогии с функцией `rescaler`.
```
tf.image.resize
```

In [ ]:
def half_size_to_full_size(scale=255):
    # your code
    pass

## Задание 1.б

Определить модель состоящую из одно слоя деконволюции с одним фильтром размера 3x3 и страйдом=2, без bias, переводящую монохромное изображение 14x14 в монохромное изображение 28x28. В качестве функции потерь используем среднеквадратичную ошибку. Опртимизатор - Adam с дефолтными настройками.
```
tf.keras.layers.Conv2DTranspose
```

In [ ]:
superresolution = # your code

# Задание 1.в

* Определить callback для использования во время обучения. По окончани каждой эпохи этот callback должен выводить все элементы фильтра деконволюции в виде чисел. 
* Обучить модель на 10 эпохах, используя `mnist_train` и `mnist_test` для валидации. В конце каждой эпохи можно контролировать элементы фильтра с помощью созданного callback-а
* Какой в результате после обучения получается фильтр? Похож ли он на
$$
\begin{bmatrix}
0.25 & 0.5 & 0.25 \\
0.5 & 1.0 & 0.5 \\
0.25 & 0.5 & 0.25
\end{bmatrix}
$$
Если нет, то почему?
```
tf.keras.callbacks.Callback
superresolution.fit(..., callbacks=[PrintKernelCallback(superresolution)])
```

In [ ]:
class PrintKernelCallback(tf.keras.callbacks.Callback):
    # your code

## Задание 1.г

Определить регуляризацию для фильтра так, чтобы в результате получалась матрица
$$
\begin{bmatrix}
0.25 & 0.5 & 0.25 \\
0.5 & 1.0 & 0.5 \\
0.25 & 0.5 & 0.25
\end{bmatrix}
$$
Создать новую модель с этой регуляризацией, обучить её и убедиться, что теперь получается правильный фильтр.
```
tf.keras.regularizers.Regularizer
```

In [ ]:
class SymRegularizer(tf.keras.regularizers.Regularizer):
    # your code

In [ ]:
superresolution_reg = # your code

# Автоэнкодер

Цель этого задания - научиться стоить и обучать простейшие автоэнкодеры.

## Задание 2.а

* Определить функцию превышения максимального уровня сигнала над средним значением шума
$$
PSNR = 10 \log_{10} \frac{\max_{ij} y_{true, ij}^2}{MSE(y_{true}, y_{pred})}
$$
* Определить функцию, переводящую пару `(X, y)`, где `X` это монохромное изображение 28x28, `y` это класс, в пару двух одинаковых изображений `(X, X)`, аналогично заданию 1.а

In [ ]:
def PSNR(y_true, y_pred):
    # your code
    pass

In [ ]:
def image_in_image_out(scale=255):
    # your code
    pass

## Задание 2.б

Определить метод, возвращающий энкодер, принимающий монохромное изображение 28x28, и выдающий латентный вектор заданной размерности. 

Архитектура энкодера:
* слой свёртки с 32 фильтрами размерности 3х3 и страйдом 2, нелинейность - relu
* слой свёртки с 64 фильтрами размерности 3х3 и страйдом 2, нелинейность - relu
* полносвязный слой с выходной размерностью равной `latent_dim` 

![Encoder](homework2/encoder.png)

In [ ]:
def get_encoder(latent_dim=10):
    # your code
    pass

## Задание 2.в

Определить метод, возвращающий декодер, принимающий латентный вектор заданной размерности и выдающий монохромное изображение 28x28.

Архитектура декодера:
* полносвязный слой, переводящий входной латентный вектор в вектор размерности 7\*7\*32
* слой, переводящий вектор размерности 7\*7\*32 в тензор размерности \[7, 7, 32\]
* слой деконволюции с 64 фильтрами размерности (3, 3), страйдом 2 и нелинейностью relu
* слой деконволюции с 32 фильтрами размерности (3, 3), страйдом 2 и нелинейностью relu
* слой деконволюции с 1 фильтром размерности (3, 3), страйдом 1 и нелинейностью sigmoid

![Decoder](homework2/decoder.png)

In [ ]:
def get_decoder(latent_dim=10):
    # your code
    pass

## Задание 2.г
* Определить класс `AutoEncoder` при инициализации принимающий 2 параметра: сеть-энкодер, и сеть-декодер, при вызове возвращающий результат последовательного применеия к изображению энкодера и декодера.
* Определить объект этого класса, модель `autoencoder`, с оптимизатором Adam(1e-4), среднеквадратичной функцией потерь, использующую метрику `PSNR`.

In [ ]:
class AutoEncoder(tf.keras.Model):
    # your code

In [ ]:
autoencoder = # your code

Отобразим пары исходное изображение - восстановленное изображение

In [ ]:
def plot_sample(model, dataset, num_examples=10, figsize=(20, 5)):
    plt.figure(figsize=figsize)
    rescaler = image_in_image_out()
    for n, (x, y) in enumerate(dataset.shuffle(1000).batch(1).take(num_examples)):
        _, x_orig = rescaler(x, y)
        plt.subplot(2, num_examples, n + 1)
        plt.imshow(x_orig[0], cmap='gray')
        plt.axis('off')
        plt.subplot(2, num_examples, n + 1 + num_examples)
        plt.imshow(model(x_orig)[0], cmap='gray')
        plt.axis('off')
    plt.show()

In [ ]:
class PlotCallback(tf.keras.callbacks.Callback):
    
    def __init__(self, model, dataset):
        self.model = model
        self.dataset = dataset
    
    def on_epoch_end(self, epoch, logs=None):
        plot_sample(self.model, self.dataset)

## Задание 2.е

Создать объект класса AutoEncoder, и обучить эту модель на 10 эпохах датасета `mnist_train`, выводя после каждой эпохи несколько примеров работы автоэнкодера. В качестве функции потерь использовать среднеквадратичную ошибку, оптимизатор - Adam со скоростью обучения 1e-4, в качестве метрики использовать созданную в задании 2.a функцию `PSNR`. Для визуального котроля качества обучения использовать `fit(..., callbacks=[PlotCallback(autoencoder, mnist_test)])`.

In [ ]:
# your code

# DeepFake

В этом задании мы реализуем знаменитую технологию замены лиц, DeepFake. Однако, вместо того, чтобы работать с настоящими лицами, что требует погружения во многие детали и больших вычислительных мощностей, мы обучим DeepFake на цифрах. Т.е. мы обучим систему, переводящую, к примеру, семерки в единицы, с сохранением некотороых характерных черт, например, наклона.

## Задание 3.а

* Определить один общий энкодер 
* Создать 2 сети с архитектурой автоэнкодера, имеющих общий энкодер и разные декодеры, функция потерь - среднеквадратичная ошибка, оптимизатор - Adam(1e-4), метрика - PSNR.

In [ ]:
encoder_common = # your code

deepfake_A = # your code

deepfake_B = # your code

## Задание 3.б

* Выбрать 2 цифры, например, 7 и 8
* Обучать попеременно первый и второй автоэнкодер по 1 эпохе на отфильтроывнных датасетах, например
    * первый автоэнкодер обучается только на цифре 7
    * второй автоэнкодер обучается только на цифре 8
* Провести процедуру попеременного обучения 10 раз, после каждого раза для визуального контроля выводить
    * результат преобразования первым автоэнкодером цифры 8 (не той, на которой он учился!)
    * результат преобразования вторым автоэнкодером цифры 7 (не той, на которой он учился!)
* Описать результат

In [ ]:
digit_A = 7
digit_B = 1
for epoch in range(10):
    # your code

## Вращаем цифры

Выполнив это код мы должны увидеть, что созданная цифра повторяет угол поворота исходной, как в настоящем deepfake

In [ ]:
def plot_rotate(model_A, model_B, digit_A, digit_B, angles=np.linspace(-0.5, 0.5, 11)):
    for x, y in mnist_test.filter(lambda x, y: y == digit_A).shuffle(1000).take(1):
        plt.figure(figsize=(20, 5))
        for n, angle in enumerate(angles):
            x_rot = tfa.image.rotate(x, angle)
            plt.subplot(2, 11, n + 1)
            plt.imshow(1 - x_rot / 255, cmap='gray')
            plt.axis('off')
            x_trans = model_A(1 - x_rot[np.newaxis, :] / 255)
            plt.subplot(2, 11, n + 1 + 11)
            plt.imshow(x_trans[0], cmap='gray')
            plt.axis('off')
        plt.show()
    for x, y in mnist_test.filter(lambda x, y: y == digit_B).shuffle(1000).take(1):
        plt.figure(figsize=(20, 5))
        for n, angle in enumerate(angles):
            x_rot = tfa.image.rotate(x, angle)
            plt.subplot(2, 11, n + 1)
            plt.imshow(1 - x_rot / 255, cmap='gray')
            plt.axis('off')
            x_trans = model_B(1 - x_rot[np.newaxis, :] / 255)
            plt.subplot(2, 11, n + 1 + 11)
            plt.imshow(x_trans[0], cmap='gray')
            plt.axis('off')
        plt.show()

In [ ]:
plot_rotate(deepfake_B, deepfake_A, digit_A, digit_B)

# DeepFake + GAN

В этом задании мы попробуем улучшить качество восстановленных изображений, добавив дискриминатор.

# Задание 4.a

Определить функцию, возвращающую дискриминатор. Дискриминатор принимает на вход монохромные изображения 28x28 и возвращает одно число в интервале от 0 до 1, которое показывает степень уверенности дискриминатора в том, что ему бло предъявлено оригинальное (не ситезированное) изображение.

Архитектура дискриминатора:
* слой свёртки с 64 фильтрами размерности (3, 3), страйдом 2 и нелинейностью relu
* слой свёртки с 32 фильтрами размерности (3, 3), страйдом 2 и нелинейностью relu
* полносвязный слой с выходной размерностью 1 и нелинейностью sigmoid

![Дискриминатор](homework2/discriminator.png)

In [ ]:
def get_discriminator():
    # your code
    pass

## Задание 4.б

Определить
* Общий энкодер
* 2 автоэнкодера, имеющих общий энкодер и разные декодеры, функция потерь - среднеквадратичная ошибка, оптимизатор - Adam(1e-4)
* 2 дискриминатора, оптимизатор - Adam(1e-4)

In [ ]:
encoder_common = # your code

dfgan_A = # your code

dfgan_B = # your code

discriminator_A = # your code

discriminator_B = # your code

Определим итераторы над датасетами

In [ ]:
digit_A = 1
digit_B = 8

iter_A = iter(mnist_train.filter(lambda x, y: y == digit_A).map(rescaler(with_label=False)).shuffle(1000).batch(32).repeat())
iter_B = iter(mnist_train.filter(lambda x, y: y == digit_B).map(rescaler(with_label=False)).shuffle(1000).batch(32).repeat())

## Задание  4.в

Определить функцию обучения для двух автоэнкодеров и двух дискриминаторов
* Автоэнкодеры учатся восстанавливать изображения, **каждый из своего класса**, например, первый учится востанавливать единицы, второй - восьмерки
* Дискриминаторы учатся отличать восстановленные изображения от оригинальных, снова каждый в своем классе
* В функции потерь дискриминаторов можно использовать `tf.keras.losses.BinaryCrossEntropy(label_smoothing=0.2)` **только для оригинальных изображений**. Реконструированные изображения используют обычную функцию потерь, без smoothing
* На время отладки аннотацию `@tf.function` можно закомментировать

In [ ]:
@tf.function
def train_step(gan_weight = 1e-3):
    
    # your code
    
    # loss_rec_A - L2 часть функции потерь для dfgan_A
    # loss_rec_B - L2 часть функции потерь для dfgan_B
    # loss_gan_A - GAN часть функции потерь для dfgan_A
    # loss_gan_B - GAN часть функции потерь для dfgan_B
    
    loss_A = loss_rec_A + gan_weight * loss_gan_A
    loss_B = loss_rec_B + gan_weight * loss_gan_B

    # loss_dA - значение функции потерь для discriminator_A
    # loss_dB - значение функции потерь для discriminator_B
    # accuracy_dA - аккуратность discriminator_A
    # accuracy_dB - аккуратность discriminator_B
    
    return loss_A, loss_B, loss_dA, loss_dB, accuracy_dA, accuracy_dB 

## Обучаем DeepFake + GAN

In [ ]:
num_steps = 1000

log_every = 10
plot_every = 200

progbar = tf.keras.utils.Progbar(num_steps)
for step in range(num_steps):
    
    loss_A, loss_B, loss_dA, loss_dB, accuracy_dA, accuracy_dB = train_step(1e-3)
    if step % log_every == 0:
        logs = [
            ('loss_A', loss_A), 
            ('loss_B', loss_B), 
            ('loss_dA', loss_dA), 
            ('loss_dB', loss_dB), 
            ('accuracy_dA', accuracy_dA), 
            ('accuracy_dB', accuracy_dB)
        ]
        progbar.update(step, values=logs)        
    if step % plot_every == 0:
        plot_sample(dfgan_A, mnist_test.filter(lambda x, y: y == num_B))
        plot_sample(dfgan_B, mnist_test.filter(lambda x, y: y == num_A))

## Вращаем цифры

In [ ]:
plot_rotate(dfgan_B, dfgan_A, digit_A, digit_B)

# Cycle GAN

Альтернативный подход к замене одних объектов другими. И снова для простоты вместо того, чтобы превращать лошадей в зебр и обратно, мы будем превращать одни цифры в другие. При этом необязательно признаки отображаются в аналогичные признаки, например, не обязательно наклон будет отображаться в наклон. Возможно, признаки будут отображаться в какие-то другие признаки. Например, наклон вправо будет отбражаться в наклон влево. Или в толщину линии, рисующей цифры.

## Задание 5.а

Определить 
* 2 сети с архитектурой автоэнкодера, будем называть их трансляторами, имеющих **различные энкодеры и различные декодеры**, оптимизатор - Adam(1e-4), эти сети должны переводить картинки из одного класса в другой и обратно
* 2 дискриминатора, оптимизатор - Adam(1e-4)

In [ ]:
translator_AB = # your code

translator_BA = # your code

discriminator_A = # your code

discriminator_B = # your code

Определим итераторы над датасетами

In [ ]:
digit_A = 4
digit_B = 7

iter_A = iter(mnist_train.filter(lambda x, y: y == digit_A).map(rescaler(with_label=False)).shuffle(1000).batch(32).repeat())
iter_B = iter(mnist_train.filter(lambda x, y: y == digit_B).map(rescaler(with_label=False)).shuffle(1000).batch(32).repeat())

## Задание 5.б

Определить функцию обучения для двух трансляторов и двух дискриминаторов
* Трансляторы учатся восстанавливать изображения, после прямого и обратного перевода, в двух возможных порядках, наример, если мы превращаем 4 в 7 и наоборот, то трансляторы должны учить восстанвливать как 4 -> 7 -> 4, так и 7 -> 4 -> 7. Функция потерь для трансляторов - L1
* Дискриминаторы учатся различать переведённое изображения от оригинальных, **каждый в своем классе**
* В функции потерь дискриминаторов можно использовать `tf.keras.losses.BinaryCrossEntropy(label_smoothing=0.2)` **только для оригинальных изображений**. Реконструированные изображения используют обычную функцию потерь, без smoothing
* На время отладки аннотацию `@tf.function` можно закомментировать

In [ ]:
@tf.function
def train_step(gan_weight = 1e-3):
    
    # your code
    
    # loss_rec_A - L2 часть функции потерь для translator_AB(translator_BA(...))
    # loss_rec_B - L2 часть функции потерь для translator_BA(translator_AB(...))
    # loss_gan_A - GAN часть функции потерь для dfgan_A
    # loss_gan_B - GAN часть функции потерь для dfgan_B
    
    loss_trans = loss_rec_A + loss_rec_B + gan_weight * (loss_gan_A + loss_gan_B)
    
    # loss_trans необходимо использовать для обучения обоих трансляторов!

    # loss_dA - значение функции потерь для discriminator_A
    # loss_dB - значение функции потерь для discriminator_B
    # accuracy_dA - аккуратность discriminator_A
    # accuracy_dB - аккуратность discriminator_B
    
    return loss_trans, loss_dA, loss_dB, accuracy_dA, accuracy_dB 

## Обучаем трансляторы (долго!)

In [ ]:
num_steps = 50000

log_every = 10
plot_every = 1000

progbar = tf.keras.utils.Progbar(num_steps)
for step in range(num_steps):
    loss_trans, loss_dA, loss_dB, accuracy_dA, accuracy_dB = train_step(gan_weight = 1e-3)
    if step % log_every == 0:
        logs = [
            ('loss_trans', loss_trans), 
            ('loss_dA', loss_dA), 
            ('loss_dB', loss_dB), 
            ('accuracy_dA', accuracy_dA), 
            ('accuracy_dB', accuracy_dB)
        ]
        progbar.update(step, values=logs)
        
    if step % plot_every == 0:
        plot_sample(translator_AB, mnist_test.filter(lambda x, y: y == num_B))
        plot_sample(translator_BA, mnist_test.filter(lambda x, y: y == num_A))

## Вращаем цифры

In [ ]:
plot_rotate(translator_BA, translator_AB, digit_A, digit_B)